# Importing modules:

In [ ]:
%matplotlib inline
# Importing sql types
from pyspark.sql.types import *
# Importing sql functions
from pyspark.sql.functions import *
# Importing plotting libraries
import matplotlib.pyplot as plt
# Importing numeric module
import numpy as np
# Importing local dataFrame library
import pandas as pd
# Importing features to build tables
from IPython.display import display, HTML
# Importing time librarie to measure time
import time
from functools import partial
import string
import unicodedata

from pyspark import SQLContext; 
# sqlContext = SQLContext(sc)

In [2]:
# This class makes an analisis of dataframe datatypes and its different general features.
class DataFrameTransformer():
    def __init__(self, df):
        # Dataframe
        self.df = df
    
    # Funtion to analize column datatypes, it also plot proportion datatypes and histograms:   
    # Transformation functions:
    # Trimming spaces in columns:
    def trimCol(self, columns):
        # Function to trim spaces in columns with strings datatype
        def colTrim(columns):
            exprs = [trim(col(c)).alias(c) if c in columns else c for (c, t) in self.df.dtypes]
            self.df = self.df.select(*exprs)

        # Asserting data variable first is integer:
        assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."

        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == 'string', self.df.dtypes)]
        
        # If None or [] is provided with column parameter:
        if (columns == "*"): columns = validCols
        
        # Columns 
        if type(columns) == type('s'): columns = [columns]
        
        assert all(col in validCols for col in columns), 'Error: Columns or column does not exist in the dataFrame or or is numberic column'
        
        # Trimming spaces in columns:
        colTrim(columns)

    # Drop
    def dropCol(self, columns):
        
        def colDrop(columns):
            exprs = filter(lambda c: c not in columns, self.df.columns)
            self.df = self.df.select(*exprs)
    
        # Asserting data variable is string or list:
        assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."
    
        # If None or [] is provided with column parameter:
        assert columns != [], "Error: Column can not be a empty list []"
        
        # Columns 
        if type(columns) == type('s'): columns = [columns]
            
        # Columns 
        assert all(col in self.df.columns for col in columns), 'Error: Columns or column does not exist in the dataFrame'
        
        # Calling colDrop function
        colDrop(columns)
        
    def replaceCol(self, search, changeTo, columns):
        
        def colReplace(columns):
            self.df = self.df.replace(search, changeTo, subset=columns)
            
        
        # Asserting columns is string or list:
        assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."
        # Asserting search parameter is a string or a number
        assert type(search) == type(' ') or type(search) == type(1.2) or type(search) == type(1), "Error: Search parameter must be a number or string"
        
        # Asserting changeTo parameter is a string or a number
        assert type(changeTo) == type(' ') or type(changeTo) == type(1.2) or type(changeTo) == type(1), "Error: changeTo parameter must be a number or string"
        
        # Asserting search and changeTo have same type
        assert type(search) == type(changeTo), 'Error: Search and ChangeTo must have same datatype: Integer, String, Float'
        
        # If None or [] is provided with column parameter:
        assert columns != [], "Error: Column can not be a empty list []"
        
        # Change 
        types = {type(''): 'string', type(int(1)): 'int', type(float(1.2)): 'float', type(1.2): 'double'};
               
        validCols = [c for (c, t) in filter(lambda t: t[1] == types[type(search)], self.df.dtypes)]     
        
        # If None or [] is provided with column parameter:
        if (columns == "*"): columns = validCols[:]
            
        # If columns is string, make a list:
        if type(columns) == type(' '): columns = [columns]
            
        # Asserting if columns provided are in dataFrame:
        assert all(col in self.df.columns for col in columns), 'Error: Columns or column does not exist in the dataFrame'
   
        # Asserting if selected column datatype and search and changeTo parameters are the same:
        assert all(column in validCols for column in  columns), 'Error: Column and Search and ChangeTo do not have same datatype'
        
        colReplace(columns)
        
    def dropRow(self, columns):
            # Function to trim spaces in columns with strings datatype
        def rowDrop(columns):
            exprs = [trim(col(c)).alias(c) if c in columns else c for (c, t) in self.df.dtypes]
            self.df = self.df.select(*exprs)

        # Asserting data variable column is string or column:
        assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."

        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == 'string', self.df.dtypes)]
        
        # If None or [] is provided with column parameter:
        if (columns == "*"): columns = validCols
        
        # Columns 
        if type(columns) == type('s'): columns = [columns]
        
        assert all(col in validCols for col in columns), 'Error: Columns or column does not exist in the dataFrame or or is numberic column'
        
        # Trimming spaces in columns:
        colTrim(columns)
        
    def deleteRow(self, func):
        self.df = self.df.filter(func)
    
        #tempFunc = lambda inColumn, inSearch: (inColumn > inSearch);  
        #delectRow(df, tempFunc(col('integers'), 7))
        
        #tempFunc = lambda inColumn, inSearchMin, inSearchMax: (inColumn > inSearchMin) & (inColumn < inSearchMax);  
        #deleteRow(df, tempFunc(col('integers'), 7, '9.0'))

    def setCol(self, columns, func, dataType):

        dictTypes = {'string': StringType(), 'str': StringType(), 'integer': IntegerType(), 
                     'int': IntegerType(), 'float': FloatType(), 'double': DoubleType(), 'Double': DoubleType()}

        Types = {'string': 'string', 'str': 'string', 'String': 'string','integer': 'int', 
                     'int': 'int', 'float': 'float', 'double': 'double', 'Double': 'double'}

        try:
            function = udf(func, dictTypes[dataType])
        except KeyError:       
            assert False, "Error, dataType not recognized"

        def colSet(columns ,function):
            exprs = [function(col(c)).alias(c) if c in columns else c for (c, t) in self.df.dtypes]
            try:
                self.df = self.df.select(*exprs)
            except:
                assert False, "Error: Make sure if operation is compatible with row datatype."

        # Asserting data variable column is string or column:
        assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."

        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == Types[dataType], self.df.dtypes)]

        # If None or [] is provided with column parameter:
        if (columns == "*"): columns = validCols[:]

        # If columns is string, make a list:
        if type(columns) == type(' '): columns = [columns]

        # Asserting if columns provided are in dataFrame:
        assert all(col in self.df.columns for col in columns), 'Error: Columns or column does not exist in the dataFrame'

        # Asserting if selected column datatype and search and changeTo parameters are the same:
        assert all(column in validCols for column in  columns), 'Error: Column and datatype parameters do not have same datatype'

        colSet(columns, function)
    
    # Drop
    def keepCol(self, columns):
        
        def colKeep(columns):
            exprs = filter(lambda c: c in columns, self.df.columns)
            self.df = self.df.select(*exprs)
    
        # Asserting data variable is string or list:
        assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."
    
        # If None or [] is provided with column parameter:
        assert columns != [], "Error: Column can not be a empty list []"
        
        # Columns 
        if type(columns) == type('s'): columns = [columns]
            
        # Columns 
        assert all(col in self.df.columns for col in columns), 'Error: Columns or column does not exist in the dataFrame'
        
        # Calling colDrop function
        colKeep(columns)
        
    def clearAccents(self, columns):
        # Asserting columns is string or list:
        assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."
        
        # If None or [] is provided with column parameter:
        assert columns != [], "Error: Column can not be a empty list []"
        
        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == 'string', self.df.dtypes)]

        # If None or [] is provided with column parameter:
        if (columns == "*"): columns = validCols[:]
            
        # If columns is string, make a list:
        if type(columns) == type(' '): columns = [columns]
        
        # Asserting if columns provided are in dataFrame:
        assert all(col in self.df.columns for col in columns), 'Error: Columns or column does not exist in the dataFrame'

        # Asserting if selected column datatype and search and changeTo parameters are the same:
        assert all(column in validCols for column in  columns), 'Error: Column and datatype parameters do not have same datatype'

        
        # Receives  a string as an argument
        def remove_accents(inputStr):
            # first, normalize strings:
            nfkdStr = unicodedata.normalize('NFKD', inputStr)
            # Keep chars that has no other char combined (i.e. accents chars)
            withOutAccents = u"".join([c for c in nfkdStr if not unicodedata.combining(c)])
            return withOutAccents

        function = udf(lambda x: remove_accents(x), StringType())
        exprs = [function(col(c)).alias(c) if c in columns else c for c in self.df.columns]
        self.df = self.df.select(*exprs)

        
    def removeSpecialChars(self, columns):
        # Asserting columns is string or list:
        assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."
        
        # If None or [] is provided with column parameter:
        assert columns != [], "Error: Column can not be a empty list []"
         
        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == 'string', self.df.dtypes)]

        # If None or [] is provided with column parameter:
        if (columns == "*"): columns = validCols[:]
            
        # If columns is string, make a list:
        if type(columns) == type(' '): columns = [columns]
        
        # Asserting if columns provided are in dataFrame:
        assert all(col in self.df.columns for col in columns), 'Error: Columns or column does not exist in the dataFrame'

        # Asserting if selected column datatype and search and changeTo parameters are the same:
        assert all(column in validCols for column in  columns), 'Error: Column and datatype parameters do not have same datatype'

        def rmSpecChars(inputStr):
            #Remove all punctuation and control characters
            for punct in (set(inputStr) & set(string.punctuation)):
                inputStr = inputStr.replace(punct,"")
            return inputStr

        # User define function that does operation in cells
        function = udf(lambda cell: rmSpecChars(cell), StringType())

        exprs = [function(col(c)).alias(c) if c in columns else c for c in self.df.columns]
        self.df = self.df.select(*exprs)
        
    def renameCol(self, column, newName):
        # Asserting columns is string or list:
        assert type(column) == type(''), "Error: Column argument must be a string"
       
        # Asserting columns is string or list:
        assert type(newName) == type(''), "Error: New column name argument must be a string or a list."
       
        # Asserting parameters are not empty strings:
        assert ((column != '') and (newName != '')), "Error: Input parameters can't be empty strings"
       
        # Asserting input column is in column of dataFrame:
        self.df = df.withColumnRenamed(column, newName)


# Cargando dataframe:

# Si se desea un data set corto de prueba:

In [100]:
import random
# Creación de un dataframe de prueba:
schema = StructType(
        [
        StructField("strings", StringType(), True),
         StructField("integers", IntegerType(), True),
        #StructField("booleans",  BooleanType(), True),
        StructField("integers2", IntegerType(), True),
        StructField("floats",  FloatType(), True),
        StructField("double",  DoubleType(), True)
        ]
)

# Cambiando una columna de strings en double
#df = df.withColumn('pepa', df.pepa + 0)

size = 200
# Generating strings column:
foods = ['    pizza!       ', 'pizza', 'PIZZA;', 'piZza', 'pízzá', 'Pizza', 'Piz;za']
foods = [foods[random.randint(0,6)] for count in range(size)]
# Generating integer column:
numCol1 = [random.randint(0,9) for number in range(size)]
# Generating boolean column:
#numCol2 = [bool(random.getrandbits(1)) for number in range(size)]
numCol2 = [random.randint(0,9) for number in range(size)]
# Generating integer column:
numCol3 = [random.random() for number in range(size)]
# Generating integer column:
numCol4 = [random.random() for number in range(size)]

# Building dataFrame:
df = sqlContext.createDataFrame(list(zip(foods, numCol1, numCol2, numCol3, numCol4)),schema=schema)
df.show()

+-----------------+--------+---------+-----------+-------------------+
|          strings|integers|integers2|     floats|             double|
+-----------------+--------+---------+-----------+-------------------+
|            Pizza|       1|        2| 0.27276418| 0.6613167128437065|
|           Piz;za|       7|        2| 0.11393974| 0.6985988822235742|
|            pízzá|       8|        6|  0.6150019|0.49384417423959026|
|            pizza|       5|        6| 0.68132496| 0.3598785838606483|
|            pizza|       3|        4|  0.1834308| 0.2925823602461035|
|            Pizza|       4|        2|  0.5925784|0.43374931785012694|
|            Pizza|       1|        8|  0.3665534| 0.9427310133178698|
|    pizza!       |       3|        9| 0.61260915|0.07517403736806638|
|    pizza!       |       5|        9| 0.67193794|0.47009181351076623|
|           Piz;za|       9|        7| 0.42232946|0.08115945367243682|
|            piZza|       6|        5| 0.33166534|0.29004299937090117|
|    p

# Ejemplos de uso:

In [ ]:
# Cargando el analizador, con el df de entrada:
analizer = DataFrameTransformer(df)
# Trim:
#analizer.trimCol(columns="*")
# Drop:
#analizer.dropCol(columns=['integers2'])
# Replace:
#analizer.replaceCol(1,1544544,"*")
# DeleteRow:
#tempFunc = lambda inColumn, inSearchMin, inSearchMax: (inColumn > inSearchMin) & (inColumn < inSearchMax);  
#deleteRow(df, tempFunc(col('integers'), 7, '9.0'))
#tempFunc = lambda inColumn, inSearch: (inColumn > inSearch);  
#analizer.deleteRow(tempFunc(col('integers'), 7))




#df = df.withColumn('Price', df.Price + 0)

#funcion = lambda cell: cell + 10045400 if (cell > 100000 ) else cell
#analizer.setCol(['Price'], funcion, 'double')

#analizer.df.select('Price').show()
#analizer.keepCol(['strings','integers'])
#analizer.df.show()


In [ ]:
#a = Analizer.DataFrameAnalizer(df, sc)
#a.columnAnalize(columnList="*")

In [ ]:
print (' Replacing a number if value in cell is greater than 5:')
# Replacing a number:   
slen = udf(lambda cell: 10000 if (cell > 5 ) else cell, IntegerType())
setCol(df, ['integers'], slen)

In [ ]:
# Number operation:
print (' Add a number to cells in columns:')
slen = udf(lambda cell: cell + 10000 , FloatType())
setCol(df, ['floats'], slen)

In [ ]:
# String manipulation:
print ('Uppercase operation in column:')
slen = udf(lambda cell: cell.upper(), StringType())
setCol(df, ['strings'], slen)

# Si se desea un data set real:

In [2]:
# Input parameters:
#pathFile = '/datasets/quiero-casa-simplificado-arg-sexo.csv' 
#delimiter_mark = ','

# filename = "/datasets/quierocasa-2.csv"
#delimiter_mark = ';'

filename = "file:///home/andrea/Downloads/QuieroCasa.csv"

#filename = 'file:///home/hreyes/pythoncleaning/cleaning-quiero-casa/datasets/world-development-indicators/Indicators.csv'
#delimiter_mark = ','
#pathFile = filename

#pathFile = 'file:///home/hreyes/pythoncleaning/cleaning-quiero-casa/datasets/prueba'
delimiter_mark = ';'
#filename = 'file:///home/hreyes/pythoncleaning/cleaning-quiero-casa/datasets/quierocasa.csv'


# Reading data:
df = sqlContext.read \
    .format('com.databricks.spark.csv') \
    .options(header='true') \
    .options(delimiter=delimiter_mark)\
    .options(inferSchema='true')\
    .load(filename) \
    .cache()

# Reading data:
#df = sqlContext.read.parquet(pathFile)
#df.printSchema()

# Frecuencia de valores

In [4]:
from pyspark.sql.functions import *
def fq(df, columns, sort_by_count = True):
    def frecuency(columns, sort_by_count):
        
        if sort_by_count:
            for column in columns:
                freq = (df
                        .groupBy(column)
                        .count()
                        .orderBy("count", ascending=False))

                if (freq.where("count > 1").count() == 0):
                    print("No existen valores que agrupar en la columna", column)
                else:
                    freq.show()
        else:
            for column in columns:
                freq = (df
                        .groupBy(column)
                        .count())

                if (freq.where("count > 1").count() == 0):
                    print("No existen valores que agrupar en la columna", column)
                else:
                    freq.show()

    # Asserting data variable is string or list:
    assert type(columns) == type('s') or type(columns) == type([]), "Error: Column argument must be a string or a list."

    # If None or [] is provided with column parameter:
    assert columns != [], "Error: Column can not be a empty list []"

    # Columns 
    if type(columns) == type('s'): columns = [columns]

    # Columns 
    assert all(col in df.columns for col in columns), 'Error: Columns or column does not exist in the dataFrame'

    frecuency(columns, sort_by_count)
    
def getFreq(df, column, sort_by_count = True):
    def frecuency(column, sort_by_count):

        if sort_by_count:
            freq = (df
                    .groupBy(column)
                    .count()
                    .orderBy("count", ascending=False))
        else:
            freq = (df
                    .groupBy(column)
                    .count())
            
        if (freq.where("count > 1").count() == 0):
            print("No existen valores que agrupar en la columna", column)
            return sqlContext.createDataFrame(sc.emptyRDD(), StructType([]))
        else:
            return freq

    # Asserting data variable is string:
    assert type(column) == type('s'), "Error: Column argument must be a string."

    # Columns 
    assert column in df.columns, 'Error: Column does not exist in the dataFrame'
    
    return frecuency(column, sort_by_count)

fq(df, ["STATE"], False)
# getFreq(df, "STATE", False).show()

+----------------+-----+
|           STATE|count|
+----------------+-----+
|Distriro Federal|  259|
|Estado de México|  810|
|     México D.F.| 2383|
|            D.F.|   66|
|     Mexico D.F.|  112|
|Distrito Federal|11523|
|DISTRITO FEDERAL|  339|
|distrito federal|   68|
+----------------+-----+



# Clustering based on Key Collision (Fingerprint and N-Gram Fingerprint)

In [5]:
from pyspark.ml.feature import NGram
from functools import reduce

class KeyCollision():
    def __init__(self, df):
        self.df = df
    
    """
    Cluster a dataframe column based on the Fingerprint algorithm
    """
    def __cluster_fingerprints__(self, column, outputCol, sort_tokens, remove_duplicates):
        def removeAndSort(string, sort_tokens, remove_duplicates):
            split_key = string.split()

            # remove duplicates, if chosen
            if remove_duplicates:
                dups_removed = set(split_key)
            else:
                dups_removed = split_key

            # sort the tokens, if chosen
            if sort_tokens:
                # sort the tokens
                sorted_split_key = sorted(dups_removed)
            else:
                sorted_split_key = dups_removed

            # join the tokens back together
            return " ".join(sorted_split_key)
        
        fCol = outputCol
        clusters = (self.df
                   .groupBy(column)
                   .count()
                   .select('count', column)
                   .withColumn(fCol, self.df[column])
                   .cache())

        
#         remove leading and trailing whitespace
#         change all characters to their lowercase representation
#         remove all punctuation and control characters
#         split the string into whitespace-separated tokens
#         sort the tokens and remove duplicates
#         join the tokens back together

        exprs = [trim(col(c)).alias(c) if c == fCol else c for c in clusters.columns]
        clusters = clusters.select(*exprs)
        
        exprs = [lower(col(c)).alias(c) if c == fCol else c for c in clusters.columns]
        clusters = clusters.select(*exprs)
        
        def rmvSpecialChars(inputStr):
            for punct in (set(inputStr) & set(string.punctuation)):
                inputStr = inputStr.replace(punct,"")
            return inputStr
            
        func = udf(lambda cell: rmvSpecialChars(cell), StringType())
        exprs = [func(col(c)).alias(c) if c == fCol else c for c in clusters.columns]
        clusters = clusters.select(*exprs)

        removeAndSortFunc = lambda cell: removeAndSort(cell, sort_tokens, remove_duplicates)
        func = udf(removeAndSortFunc, StringType())
        exprs = [func(col(c)).alias(c) if c == fCol else c for c in clusters.columns]
        clusters = clusters.select(*exprs)
        
        clusters.unpersist()
        return clusters
    
    """
    Cluster a DataFrame column based on the N-Gram Fingerprint algorithm
    """
    def __cluster_ngram__(self, column, n_size, outputCol):
        def joinUnique(seq):
            seen = set()
            x = [x for x in seq if not (x in seen or seen.add(x))]
            return "".join(x).replace(' ', '')
        
        newColumn = column + "Changes"
        nGramCol = outputCol
        
        clusters = (self.df.select(column)
                   .groupBy(column)
                   .count()
                   .withColumn(newColumn, self.df[column])
                   .cache())
        
#/         change all characters to their lowercase representation
#/         remove all punctuation, whitespace, and control characters
#         obtain all the string n-grams
#         sort the n-grams and remove duplicates
#         join the sorted n-grams back together

        exprs = [lower(col(c)).alias(c) if c == newColumn else c for c in clusters.columns]
        clusters = clusters.select(*exprs)
        
        def rmvSpecialChars(inputStr):
            for punct in (set(inputStr) & set(string.punctuation)):
                inputStr = inputStr.replace(punct,"")
            return "".join(inputStr.split())
            
        func = udf(lambda cell: rmvSpecialChars(cell), StringType())
        exprs = [func(col(c)).alias(c) if c == newColumn else c for c in clusters.columns]
        clusters = clusters.select(*exprs)
        
        exprs = [trim(col(c)).alias(c) if c == newColumn else c for c in clusters.columns]
        clusters = clusters.select(*exprs)
                
        # create NGram object
        ngram = NGram(n=n_size, inputCol=newColumn, outputCol=nGramCol)
        
        # flatten newColumn string values and ngram transform the column
        exprs = [split(col(c), '').alias(c) if c == newColumn else c for c in clusters.columns]
        ngram_df = ngram.transform(clusters.select(*exprs))
        
        # sort and remove duplicate ngrams
        clusters = (ngram_df
                    .select(*['count', column, sort_array(nGramCol)
                              .alias(nGramCol)]))
        
        # join sorted, unique ngrams back together
        function = udf(joinUnique, StringType())
        exprs = [function(col(c)).alias(c) if c == nGramCol else c for c in clusters.columns]
        clusters= (clusters.select(*exprs))
        
        clusters.unpersist()
        return clusters
        
    """
    Returns a clustered dataframe column as a dataframe with columns [strings = String, fingerprints = String]
    """
    def fingerprint(self, column, outputCol = "Fingerprint", sort_tokens=False, remove_duplicates=False):
        # Asserting data variable is string or list:
        assert type(column) == type('s'), "Error: Column argument must be a string."

        # If None or [] is provided with column parameter:
        assert column != "", "Error: Column can not be a empty string"

        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == 'string', self.df.dtypes)]

        # asserts the column parameter is a string column of the dataFrame
        assert column in validCols, 'Error: Columns or column does not exist in the dataFrame or is numberic column'       

        c = self.__cluster_fingerprints__(column, outputCol, sort_tokens, remove_duplicates)
        return c
    
    """
    Returns a clustered dataframe column as a dataframe with columns [strings = String, nGrams = String]
    """
    def nGramFingerprint(self, column, n_size=2, outputCol="nGram"):
        # Asserting data variable is string or list:
        assert type(column) == type('s'), "Error: Column argument must be a string."

        # If None or [] is provided with column parameter:
        assert column != "", "Error: Column can not be a empty string"

        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == 'string', self.df.dtypes)]

        # asserts the column parameter is a string column of the dataFrame
        assert column in validCols, 'Error: Columns or column does not exist in the dataFrame or is numberic column'       

        c = self.__cluster_ngram__(column, n_size, outputCol)
        return c
    
    """
    Returns the clustered dataframe column as a Python list to provide iterative properties
    """
    def fingerprintList(self, column, outputCol = 'Fingerprint', sort_tokens=False, remove_duplicates=False):
        def list_by_fingerprint(column, outputCol, sort_tokens, remove_duplicates):
            clusterDF = self.__cluster_fingerprints__(column, outputCol, sort_tokens, remove_duplicates)
            
#             (clusterDF.select('*')
#                 .join(clusterDF
#                       .groupBy('fingerprint')
#                       .agg(max('count')
#                           .alias('count'))
#                       , ['count', 'fingerprint']))
            
            # crea un mapa con el formato: {ngram: [raw strings]}

            def ext(a, b): return a + [b]
            def listify(a): return [a]
            
            return (clusterDF.map(lambda r: r[outputCol])
                       .zip(clusterDF.map(lambda f: f[column]))
                       .combineByKey(listify, ext, ext)
                       .collectAsMap())
    
        # Asserting data variable is string or list:
        assert type(column) == type('s'), "Error: Column argument must be a string."

        # If None or [] is provided with column parameter:
        assert column != "", "Error: Column can not be a empty string"

        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == 'string', self.df.dtypes)]

        # asserts the column parameter is a string column of the dataFrame
        assert column in validCols, 'Error: Columns or column does not exist in the dataFrame or is numberic column'       

        c = list_by_fingerprint(column, outputCol, sort_tokens, remove_duplicates)
        return c
    
    """
    Returns the clustered dataframe column as a Python list to provide iterative properties
    """
    def nGramFingerprintList(self, column, n_size=2, outputCol='nGram'):
        def list_by_fingerprint(column, n_size, outputCol):
            clusterDF = self.__cluster_ngram__(column, n_size, outputCol)
            
            # crea un mapa con el formato: {ngram: [raw strings]}

            def ext(a, b): return a + [b]
            def listify(a): return [a]
            
            return (clusterDF.map(lambda r: r[outputCol])
                       .zip(clusterDF.map(lambda f: f[column]))
                       .combineByKey(listify, ext, ext)
                       .collectAsMap())
    
        # Asserting data variable is string or list:
        assert type(column) == type('s'), "Error: Column argument must be a string."

        # If None or [] is provided with column parameter:
        assert column != "", "Error: Column can not be a empty string"

        # Filters all string columns in dataFrame
        validCols = [c for (c, t) in filter(lambda t: t[1] == 'string', self.df.dtypes)]

        # asserts the column parameter is a string column of the dataFrame
        assert column in validCols, 'Error: Columns or column does not exist in the dataFrame or is numberic column'  
        
        c = list_by_fingerprint(column, n_size, outputCol)
        return c

In [6]:
test = KeyCollision(df)
test.fingerprint('STATE').show()
test.fingerprintList('UBICAHORI')
# dict(test.fingerprintList('UBICAHORI'))
# test.nGramFingerprint('STATE', 1).show()
# print(test.nGramFingerprintList('STATE', 1))

+-----+----------------+----------------+
|count|           STATE|     Fingerprint|
+-----+----------------+----------------+
|  259|Distriro Federal|distriro federal|
|  810|Estado de México|estado de méxico|
| 2383|     México D.F.|       méxico df|
|   66|            D.F.|              df|
|  112|     Mexico D.F.|       mexico df|
|11523|Distrito Federal|distrito federal|
|  339|DISTRITO FEDERAL|distrito federal|
|   68|distrito federal|distrito federal|
+-----+----------------+----------------+



{'': [''],
 'abajo con eleva autos': ['Abajo con eleva autos'],
 'abajo sin eleva autos': ['Abajo sin eleva autos'],
 'adelante': ['Adelante'],
 'adelante doble': ['Adelante doble', 'adelante doble'],
 'adelante simple': ['Adelante simple'],
 'adelante triple': ['Adelante triple', 'Adelante Triple', 'adelante triple'],
 'arriba con eleva autos': ['Arriba con eleva autos'],
 'atrás doble': ['Atrás doble', 'atrás doble'],
 'atrás simple': ['Atrás simple'],
 'atrás triple': ['Atrás triple', 'atrás triple'],
 'cajon virtual': ['Cajon virtual', 'cajon virtual', 'Cajon Virtual'],
 'en medio': ['En medio'],
 'en medio simple': ['En medio simple'],
 'en medio triple': ['En medio triple', 'En medio Triple', 'en medio triple'],
 'único': ['Único', 'único', 'ÚNICO']}

# Outlier detection based on MAD

In [3]:
import numpy as np
from numpy import floor
from pyspark.sql.functions import col, abs
import time

class OutlierDetector():
    def __init__(self, df):
        self.df = df
        
        
    def median(self, df, column):
        
        # Asserting column is string:
        assert type(column) == type('s'), "Error: Column argument must be a string."
        # Asserting if column provided is in dataFrame:
        assert column in self.df.columns, 'Error: Column does not exist in the dataFrame'
        
        p=0.5

        n = df.count()
        h = (n - 1) * p
        
        df.registerTempTable("table")

        return (sqlContext
                .sql("SELECT \
                        ROUND( \
                        PERCENTILE_APPROX(" + column + ", 0.5), 2) \
                        AS " + column + " FROM table")
                .map(lambda x: x[column])
                .first())
        
    def mad(self, column, sample=None, seed=None):
        """
        Computes median absolute deviation for the numeric column 'column' in the dataframe 'df'
        """
        self.column = column
        
        # Asserting column is string:
        assert type(column) == type('s'), "Error: Column argument must be a string."
        # Asserting if column provided is in dataFrame:
        assert column in self.df.columns, 'Error: Column does not exist in the dataFrame'

        assert sample is None or 0 < sample <= 1

        seed = seed if seed is not None else time.time()
        self.df = self.df if sample is None else df.sample(False, sample, seed)

        # MAD = median of the absolute deviations from the median
        # steps:
        # 1) calculate median 
        # 2) calculate difference of all values from the median
        # 3) create rdd containing the absolute deviation 
        # 3) calculate median of this rdd

        self.medianValue = self.median(self.df, column)

        absoluteDeviation = (self.df
                             .select(column)
                             .orderBy(column)
                             .withColumn(column, abs(col(column) - self.medianValue))
                             .cache())

        self.madValue = self.median(absoluteDeviation, column)
        
        return self.madValue
        
        
    def rangeLimits(self, threshold=2):
        """
        Returns an array with the limits of the range of accepted values, that is,
        the range given by [median-constant*MAD, median+constant*MAD]
        """
        
        if not(hasattr(self, 'column')):
            # if mad hasn't been calculated
            assert False, ("Cannot get values within range if MAD value has not been computed.\n" 
                           "Please run OutlierDetector.mad(column, sample=None, seed=None)")

        self.limits = []
        self.limits.append(round((self.medianValue - threshold*self.madValue), 2))
        self.limits.append(round((self.medianValue + threshold*self.madValue), 2))
        
        return self.limits

    def run(self, threshold=2):
        """
        Get list of values within accepted range, without duplicates
        """
        
        if not(hasattr(self, 'column')):
            # if mad hasn't been calculated
            assert False, ("Cannot get values within range if MAD value has not been computed.\n" 
                           "Please run OutlierDetector.mad(column, sample=None, seed=None)")
        else:
            if not(hasattr(self, 'limits')):
                # if limits have not been calculated but mad has
                self.rangeLimits(threshold)
                
            
        limits = self.limits
        column = self.column

        self.valuesWithinRange = list(set((self.df
                                           .map(lambda x: x[column])
                                           .filter(lambda x: x>=limits[0] and x<=limits[1])
                                           .collect())))
        
        return self.valuesWithinRange
    
    
    def outliers(self, threshold=2):
        """
        Get list of values within accepted range, without duplicates
        """
        
        if not(hasattr(self, 'column')):
            # if mad hasn't been calculated
            assert False, ("Cannot get values within range if MAD value has not been computed.\n" 
                           "Please run OutlierDetector.mad(column, sample=None, seed=None)")
        else:
            if not(hasattr(self, 'limits')):
                # if limits have not been calculated but mad has
                self.rangeLimits(threshold)
                
            
        limits = self.limits
        column = self.column

        self.valuesWithoutRange = list(set((self.df
                                            .map(lambda x: x[column])
                                            .filter(lambda x: x<limits[0] or x>limits[1])
                                            .collect())))
        
        return self.valuesWithoutRange

    def deleteOutliers(self, threshold=2):
        """
        Deletes all rows where values in the column are outliers
        """
        
        if not(hasattr(self, 'column')):
            # if mad hasn't been calculated
            assert False, ("Cannot get values within range if MAD value has not been computed.\n" 
                           "Please run OutlierDetector.mad(column, sample=None, seed=None)")
        else:
            if not(hasattr(self, 'limits')):
                # if limits have not been calculated but mad has
                self.rangeLimits(threshold)
            
        limits = self.limits
        column = self.column
        
        func = lambda x: (x>=limits[0]) & (x<=limits[1])
        self.df = self.df.filter(func(col(column)))
        
        return self.df
    
    def replaceOutliers(self, replacement, dataType, threshold=2):
        """
        Replaces outliers value with 'replacement' of a specified 'dataType' (string)
        """
        
        dictTypes = {'string': StringType(), 'str': StringType(), 'StringType()': StringType(), 
                     'integer': IntegerType(), 'int': IntegerType(), 'IntegerType()': IntegerType(),
                     'float': FloatType(), 'FloatType()': FloatType(),
                     'double': DoubleType(), 'Double': DoubleType(), 'DoubleType()': DoubleType()}
        
        if not(hasattr(self, 'column')):
            # if mad hasn't been calculated
            assert False, ("Cannot get values within range if MAD value has not been computed.\n" 
                           "Please run OutlierDetector.mad(column, sample=None, seed=None)")
        else:
            if not(hasattr(self, 'limits')):
                # if limits have not been calculated but mad has
                self.rangeLimits(threshold)
            
        limits = self.limits
        column = self.column
        
        func = udf(lambda x: replacement if (x<limits[0]) | (x>limits[1]) else x,
                  dictTypes[dataType])
        exprs = [func(col(c)).alias(c) if c is column else c for c in self.df.columns]
        self.df = self.df.select(*exprs)
        
        return self.df

In [4]:
detector = OutlierDetector(df)
# print("Original dataframe (unchanged)")
# df.show()
median = detector.median(df, 'ZIPCODE')
# median = detector.median(df, 'double')
print("\nMedian: ", median)
print("Median absolute deviation: ", detector.mad('ZIPCODE'))
print("\nRange limits: ", detector.rangeLimits(2))
# print("\nValues within accepted range: ", detector.run())
# print("\nOutlier values: ", detector.outliers())


Median:  6820.0
Median absolute deviation:  2700.0

Range limits:  [1420.0, 12220.0]


In [110]:
# print(df.dtypes)

In [54]:
detector.replaceOutliers(0, 'int')
detector.df.select('ZIPCODE').show()
# Rows with col = 0 would be deleted if OutlierDetector.deleteOutliers() would've been run instead.
# A suggested option for a real user is to use the output of OutlierDetector.median(df, column

+-------+
|ZIPCODE|
+-------+
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
|   7730|
+-------+
only showing top 20 rows



# Clustering based on Levenshtein Distance

In [108]:
from operator import add

class Levenshtein():
    def __init__(self, df):
        self.df = df
    
    """
    Performs clustering on a string column 'column' based on the levenshtein distance
    between all values
    """
    def cluster_leven(self, column, distance):
        keyer = KeyCollision(self.df)
        
        first_col = 'fingerprint'
        second_col = '_fingerprint_'
        distance_col = 'levenshtein'
        
        fdf = keyer.fingerprint(column, first_col)
        fdict = dict(keyer.fingerprintList(column, first_col))
        
        fingerprints = (fdf.select(first_col)
                        .distinct()
                        .cache())
        
        fingerprint_count = (fdf
                              .groupBy('fingerprint')
                              .agg(sum('count')
                                  .alias('count')))
        
        # computes cartesian product and get combinations between values
        # and obtains a distance matrix based on the levenshtein distance
       
        condition = first_col + '!=' + second_col
        l_matrix = (fingerprints.select(first_col)
                    .join(fingerprints.select(fingerprints[first_col].alias(second_col)), how='outer')
                    .distinct()
                    .filter(condition)
                    .withColumn('levenshtein', levenshtein(first_col, second_col))
                    .cache())
        
        clusters = {} # dict to contain clusters
        cid = 0       # indexed cluster id for iterating over clusters
        pair = []     # pair of elements with levenshtein < distance
        smallest = 0  # smallest distance found in the DF
        
        # get the smallest distance (as an int) present in the matrix 
        smallest = l_matrix.map(lambda d: d[distance_col]).min()

        # cluster procedure, clustering will be perfomed over rows with levenshtein < distance
        while (smallest < distance):
            
            grouped = False # flag to determine if pair has been added to a cluster

            # retrieves the first combination of strings with the smalles distance as a list
            first = l_matrix.filter(l_matrix[distance_col] == smallest).first()
            pair = [first[0], first[1]]

            # iterate over existing clusters
            for cid in clusters:
                
                # if any of the items in pair is in any of the clusters, 
                # the other item should be added to the list
                if any(raw in clusters[cid] for fingerprint in pair for raw in fdict[fingerprint]):
                    clusters[cid].extend(raw 
                                         for fingerprint in pair 
                                         for raw in fdict[fingerprint] 
                                         if raw not in clusters[cid])
                    grouped = True
                    break;

            # if pair was not added to a cluster, create a new one and add both elements
            if not grouped:
                cid = len(clusters)
                clusters[cid] = []
                clusters[cid].extend(raw 
                                     for fingerprint in pair 
                                     for raw in fdict[fingerprint])                 
                    
            # index of item with smallest 'count' on fingerprint_count
            dropped = 1
            
            # filter to remove all columns that include the value that will be dropped
            l_matrix = (l_matrix
                        .filter(l_matrix[first_col] != pair[dropped])
                        .filter(l_matrix[second_col] != pair[dropped]))
            
            smallest = l_matrix.map(lambda d: d[distance_col]).min()
            
        l_matrix.unpersist()
        fingerprints.unpersist()
        return clusters

In [109]:
leven = Levenshtein(df)
c = leven.cluster_leven('STATE', 10)

In [110]:
print(c)

{0: ['México D.F.', 'Mexico D.F.', 'D.F.'], 1: ['Distriro Federal', 'Distrito Federal', 'DISTRITO FEDERAL', 'distrito federal']}


In [113]:
sqlContext.clearCache()